In [1]:
# prompt: mount drive for specific folder in drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "/content/drive/MyDrive/AWS_AI_ML_Scholar"


Mounted at /content/drive
/content/drive/MyDrive/AWS_AI_ML_Scholar


In [2]:
import argparse                                #DEVELOPER : Abdul Basit
import torch
from torch.autograd import Variable
from torchvision import transforms, models
import torch.nn.functional as F
import numpy as np
from PIL import Image
import json
import os
import random
import warnings
import PIL
import math


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_dir', dest="save_dir", action="store", default="./checkpoint.pth")
    parser.add_argument('--top_k', dest='top_k', default=3)
    parser.add_argument('--filepath', dest='filepath', default='flower_data/test/16/image_06657.jpg') # use a deafault filepath to a primrose image
    parser.add_argument('--category_names', dest='category_names', default='cat_to_name_.json')
    parser.add_argument('--gpu', action='store', default='gpu')
    args = parser.parse_args(args=[])
    return args


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# def predict(image_path, model,device, topk=5):
    # ''' Predict the class (or classes) of an image using a trained deep learning model.
    # '''

    # model.to(device)
    # model.eval();
    # # Convert image from numpy to torch
    # torch_image = torch.from_numpy(np.expand_dims(process_image(image_path), axis=0)).type(torch.FloatTensor).to(device)

    # # Find probabilities (results) by passing through the function (note the log softmax means that its on a log scale)
    # log_probs = model.forward(torch_image)

    # # Convert to linear scale
    # linear_probs = torch.exp(log_probs)

    # # Find the top 3 results
    # top_probs, top_labels = linear_probs.topk(topk)
    # top_probs = np.array(top_probs.detach().cpu())[0]  # Move to CPU before converting to NumPy
    # top_labels = np.array(top_labels.detach().cpu())[0]
    # with open('cat_to_name.json', 'r') as f:
    #     cat_to_name = json.load(f)

    # # Convert to classes
    # idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    # top_labels = [idx_to_class[lab] for lab in top_labels]
    # top_flowers = [cat_to_name[lab] for lab in top_labels]

    # return top_probs, top_labels, top_flowers

def predict_custom(image_path, model, device, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    This is a customized version of the predict function with a different structure.
    '''

    # Move the model to the specified device
    model.to(device)
    model.eval()

    # Preprocess the image and convert it to a PyTorch tensor
    processed_image = torch.from_numpy(np.expand_dims(process_image(image_path), axis=0)).type(torch.FloatTensor).to(device)

    # Forward pass to get the log probabilities
    log_probs = model(processed_image)

    # Convert log probabilities to linear scale
    linear_probs = torch.exp(log_probs)

    # Get the top probabilities and corresponding labels
    top_probability, top_labels = linear_probs.topk(topk)
    top_probability = np.array(top_probability.detach().cpu())[0]
    top_labels = np.array(top_labels.detach().cpu())[0]

    # Load the mapping from category index to category name
    with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

    # Map indices to class labels
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    top_labels = [idx_to_class[lab] for lab in top_labels]

    # Map class labels to flower names
    top_flowers = [cat_to_name[lab] for lab in top_labels]

    return top_probability, top_labels, top_flowers



# Scales, crops, and normalizes a PIL image for a PyTorch model,returns an Numpy array
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # TODO: Process a PIL image for use in a PyTorch model
        #size = 256, 256
    #loading image
    im = PIL.Image.open(image)
    #original size
    width, height = im.size

    if width > height:
        height = 256

    else:
        width = 256
    im.thumbnail ((width,50000), Image.ANTIALIAS)
    #new size of im
    width, height = im.size
    #crop 224x224 in the center
    reduce = 224
    left = (width - reduce)/2
    top = (height - reduce)/2
    right = left + 224
    bottom = top + 224
    im = im.crop ((left, top, right, bottom))

    #preparing numpy array
    #to make values from 0 to 1
    numpy_img = np.array(im)/255
    # Normalize each color channel
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    numpy_img = (numpy_img-mean)/std

    numpy_img= numpy_img.transpose ((2,0,1))
    return numpy_img



def print_probability(probs, flowers):
    #Converts two lists into a dictionary to print on screen
    for i, j in enumerate(zip(flowers, probs)):
        print ("Rank {}:".format(i+1),
               "Flower: {}, liklihood: {}%".format(j[1], math.ceil(j[0]*100)))


# Loading the trained model
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model = models.vgg19(pretrained=True)
    model.name = "vgg19"
    for param in model.parameters():
        param.requires_grad = False

    # Load from checkpoint
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['mapping']
    return model


def main():
  args = parse_args()
  args.category_names = "cat_to_name.json"
  with open(args.category_names, 'r') as f:
        cat_to_name = json.load(f)
  #load the trained models
  model = load_checkpoint(args.save_dir)
  image_tensor = process_image(args.filepath)
  image_tensor = torch.from_numpy(image_tensor)
  image_tensor = image_tensor.to(device)
  #checkng for available device

  #getting the predictions
  top_probability, top_labels, top_flowers = predict_custom(args.filepath,model,device,args.top_k)
  print(top_probability)
  print_probability(top_flowers, top_probability)
  warnings.filterwarnings('ignore')

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 88.5MB/s]
<ipython-input-2-638cd4b51c76>:114: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im.thumbnail ((width,50000), Imag

[0.2986687  0.1407441  0.09940755]
Rank 1: Flower: watercress, liklihood: 30%
Rank 2: Flower: canna lily, liklihood: 15%
Rank 3: Flower: corn poppy, liklihood: 10%


In [6]:
import argparse  # DEVELOPER: Abdul Basit
import json
import math
import numpy as np
import os
from PIL import Image
import random
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import transforms, models
import warnings


def main():
    # Parse command line arguments
    parser = argparse.ArgumentParser()
    parser.add_argument('--save_dir', dest="save_dir", action="store", default="./checkpoint.pth")
    parser.add_argument('--top_k', dest='top_k', default=3)
    parser.add_argument('--filepath', dest='filepath', default='flower_data/test/16/image_06657.jpg')  # use a default filepath to a primrose image
    parser.add_argument('--category_names', dest='category_names', default='cat_to_name_.json')
    parser.add_argument('--gpu', action='store', default='gpu')
    args = parser.parse_args(args=[])

    args.category_names = "cat_to_name.json"
    with open(args.category_names, 'r') as f:
        cat_to_name = json.load(f)

    # Load the trained model
    device = check_gpu(gpu_arg=args.gpu)
    model = load_checkpoint(args.save_dir)

    # Preprocess the image
    image_tensor = process_image(args.filepath)
    image_tensor = torch.from_numpy(image_tensor)
    image_tensor = image_tensor.to(device)

    # Get predictions
    top_probs, top_labels, top_flowers = predict(args.filepath, model, device, args.top_k)
    print("Top probabilities:", top_probs)
    print("Top labels and flowers:")
    print_probability(top_flowers, top_probs)
    warnings.filterwarnings('ignore')


def gpu_check(arg_gpu):
    """
    Check for GPU availability and set the device accordingly.
    """
    if not arg_gpu or arg_gpu == "cpu":
        return torch.device("cpu")
    gpu_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    return gpu_device


def preprocess_image(image_path):
    '''
    Preprocesses a PIL image for a PyTorch model,
    returns a NumPy array
    '''
    # Load image
    image = Image.open(image_path)

    # Resize image while maintaining aspect ratio
    width, height = image.size
    if width > height:
        height = 256
    else:
        width = 256
    image.thumbnail((width, 50000), Image.ANTIALIAS)

    # Crop center to 224x224
    reduce = 224
    left = (width - reduce) / 2
    top = (height - reduce) / 2
    right = left + 224
    bottom = top + 224
    image = image.crop((left, top, right, bottom))

    # Convert image to numpy array and normalize
    numpy_img = np.array(image) / 255
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    numpy_img = (numpy_img - mean) / std

    # Transpose dimensions
    numpy_img = numpy_img.transpose((2, 0, 1))

    # Print processed image dimensions
    print("Processed image dimensions:", numpy_img.shape)

    return numpy_img


def make_prediction(image_path, model, device, topk=5):
    ''' Make a prediction for an image using a trained deep learning model.
    '''
    model.to(device)
    model.eval()
    print('Using device:', device)

    # Convert image from numpy to torch
    torch_image = torch.from_numpy(np.expand_dims(preprocess_image(image_path), axis=0)).type(torch.FloatTensor).to(device)

    # Find probabilities (results) by passing through the model
    log_probs = model.forward(torch_image)

    # Convert to linear scale
    linear_probs = torch.exp(log_probs)

    # Find the top results
    top_probs, top_labels = linear_probs.topk(topk)
    top_probs = np.array(top_probs.detach().cpu())[0]  # Move to CPU before converting to NumPy
    top_labels = np.array(top_labels.detach().cpu())[0]

    # Load class names
    with open('cat_to_name.json', 'r') as f:
        cat_to_name = json.load(f)

    # Convert labels to class names
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    top_labels = [idx_to_class[lab] for lab in top_labels]
    top_flowers = [cat_to_name[lab] for lab in top_labels]

    return top_probs, top_labels, top_flowers


def display_probabilities(probs, flowers):
    # Convert two lists into a dictionary to print on screen
    for i, j in enumerate(zip(flowers, probs)):
        print("Rank {}:".format(i + 1),
              "Flower: {}, likelihood: {}%".format(j[1], math.ceil(j[0] * 100)))

def load_checkpoint(checkpoint_path):
    # Load the model checkpoint and return the model
    checkpoint = torch.load(checkpoint_path)
    model = models.vgg19(pretrained=True)
    model.name = "vgg19"
    for param in model.parameters():
        param.requires_grad = False

    # Load from the checkpoint
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['mapping']
    return model


if __name__ == "__main__":
    main()


Device  cuda:0
Top probabilities: [0.2986687  0.1407441  0.09940755]
Top labels and flowers:
Rank 1: Flower: watercress, liklihood: 30%
Rank 2: Flower: canna lily, liklihood: 15%
Rank 3: Flower: corn poppy, liklihood: 10%
